In [1]:
from Bio.Seq import Seq
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import numpy as np
import re
import os
import gzip

In [24]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
i5_index = {
    "D501": "AGGCTATA",
    "D502": "GCCTCTAT",
    "D503": "AGGATAGG",
    "D504": "TCAGAGCC",
    "D505": "CTTCGCCT",
    "D506": "TAAGATTA",
    "D507": "ACGTCCTG",
    "D508": "GTCAGTAC"
}

for k, v in i5_index.items():
    i5_index[k] = str(Seq(v).reverse_complement())
    
i7_index = {
    "D701": "ATTACTCG",
    "D702": "TCCGGAGA", 
    "D703": "CGCTCATT", 
    "D704": "GAGATTCC", 
    "D705": "ATTCAGAA",
    "D706": "GAATTCGT",
    "D707": "CTGAAGCT",
    "D708": "TAATGCGC",
    "D709": "CGGCTATG",
    "D710": "TCCGCGAA",
    "D711": "TCTCGCGC",
    "D712": "AGCGATAG"
}

In [11]:
# BPS double barcode cross validation
dual_indices = [i7_index["D701"] + "_" +i5_index["D501"],
               i7_index["D702"] + "_" +i5_index["D502"],
               i7_index["D703"] + "_" +i5_index["D503"],
               i7_index["D704"] + "_" +i5_index["D504"],
               i7_index["D705"] + "_" +i5_index["D505"], 
               i7_index["D706"] + "_" +i5_index["D506"],
               i7_index["D707"] + "_" +i5_index["D507"],
               i7_index["D708"] + "_" +i5_index["D508"],
               i7_index["D712"] + "_" +i5_index["D508"],
               i7_index["D711"] + "_" +i5_index["D507"],
               i7_index["D710"] + "_" +i5_index["D506"],
               i7_index["D709"] + "_" +i5_index["D505"],
               i7_index["D708"] + "_" +i5_index["D504"],
               i7_index["D707"] + "_" +i5_index["D503"],
               i7_index["D706"] + "_" +i5_index["D502"],
               i7_index["D705"] + "_" +i5_index["D501"]
               ]



In [12]:
for i in dual_indices:
    vars()[i] = open(i + ".txt", "w")
f_file = FastqGeneralIterator(gzip.open("./Undetermined_S0_L001_R1_001.fastq.gz", "rt"))
r_file = FastqGeneralIterator(gzip.open("./Undetermined_S0_L001_R2_001.fastq.gz", "rt"))
seqtag_pos = 0
f_multitag_pos = 8
f_barcode_pos = 57
r_multitag_pos = 7
r_barcode_pos = 32
f_barcode_length = 34
r_barcode_length = 27

recipient_re = re.compile('\D*?(.ACA|G.CA|GA.A|GAC.)\D{4,7}?AA\D{4,7}?AA\D{4,7}?TT\D{4,7}?(.TCG|C.CG|CT.G|CTC.)\D*')
donor_re = re.compile('\D*?(.GGC|T.GC|TG.C|TGG.)\D{4,7}?AA\D{4,7}?TT\D{4,7}?(.CGG|G.GG|GC.G|GCG.)\D*')
recipient_f_clipper = re.compile('(.ACA|G.CA|GA.A|GAC.)')
donor_f_clipper = re.compile('(.GGC|T.GC|TG.C|TGG.)')
recipient_r_clipper = re.compile('(.CTC|G.TC|GC.C|GCT.)')
donor_r_clipper = re.compile('(.GCG|G.CG|GG.G|GGC.)')

min_qs = 25
quality_bps_count = 0

for f_record, r_record in zip(f_file, r_file):
    
    fr = f_record[1]
    rr = r_record[1]
    fq = [ord(i) -33 for i in list(f_record[2])]
    rq = [ord(i) - 33 for i in list(r_record[2])]
    recipient_tag_grep = recipient_re.match(str(fr)[f_barcode_pos : f_barcode_pos + f_barcode_length])
    donor_tag_grep = donor_re.match(str(rr)[r_barcode_pos : r_barcode_pos + r_barcode_length])
    
    if recipient_tag_grep is not None and donor_tag_grep is not None:
        if np.mean(fq[recipient_tag_grep.start() : recipient_tag_grep.end()]) >= min_qs and \
        np.mean(rq[donor_tag_grep.start() : donor_tag_grep.end()]) >= min_qs:
            index_read = f_record[0][-17:].replace("+", "_")
            f_multitag = fr[f_multitag_pos : f_multitag_pos + 6]
            f_seqtag = fr[0:8]
            r_seqtag = rr[0:7]
            r_multitag = rr[r_multitag_pos : r_multitag_pos + 9]
            quality_bps_count += 1
            if index_read in dual_indices:
                vars()[index_read].write(recipient_tag_grep.group()[4:-4] + "," + donor_tag_grep.group()[4:-4] 
                                     + "," + f_multitag + "," + r_multitag + "," + f_seqtag + r_seqtag + "\n")
#             vars()[index_read].write(rr + "\n")


for i in dual_indices:
    vars()[i].close()

In [14]:
print(quality_bps_count)

2377882


In [20]:
oligo_indices = dual_indices[0:8]
even_assembly_order = [oligo_indices[i] for i in [2,3,6,7]]
odd_assembly_order = [oligo_indices[i] for i in [0,1,4,5]]

In [43]:
for i in oligo_indices:
    vars()[i] = open(i + ".txt", "w")
f_file = FastqGeneralIterator(gzip.open("./Undetermined_S0_L001_R1_001.fastq.gz", "rt"))
r_file = FastqGeneralIterator(gzip.open("./Undetermined_S0_L001_R2_001.fastq.gz", "rt"))
seqtag_pos = 0
f_multitag_pos = 8
f_barcode_pos = 57
r_multitag_pos = 7
r_oligo_pos = 48
f_barcode_length = 34
f_oligo_pos_odd = 200
f_oligo_pos_even = 200
f_oligo_pos_odd_1st = 241
entry_homology_front_pos = 197
entry_homology_length = 40
r_oligo_length = 244

min_qs = 25
quality_bps_barcode_count = 0
quality_f_partial_oligo_count = 0
quality_r_full_oligo_count =0

bps_barcode_re = re.compile('\D*?(.ACA|G.CA|GA.A|GAC.)\D{4,7}?AA\D{4,7}?AA\D{4,7}?TT\D{4,7}?(.TCG|C.CG|CT.G|CTC.)\D*')
bps_barcpde_f_clipper = re.compile('(.ACA|G.CA|GA.A|GAC.)')
bps_barcpde_r_clipper = re.compile('(.CTC|G.TC|GC.C|GCT.)')
f_oligo_f_clipper = re.compile('(.GCC|C.CC|CG.C|CGC.)')
r_oligo_f_clipper = re.compile('(.CGC|C.GC|CC.C|CCG.)')
entry_homology_front = re.compile('(.GAG|C.AG|CG.G|CGA.)')
entry_homology_end = re.compile('(.CGT|G.GT|GC.T|GCG.)')

for f_record, r_record in zip(f_file, r_file):
    
    fr = f_record[1]
    rr = r_record[1]
    fq = [ord(i) -33 for i in list(f_record[2])]
    rq = [ord(i) - 33 for i in list(r_record[2])]
    recipient_tag_grep = bps_barcode_re.match(str(fr)[f_barcode_pos : f_barcode_pos + f_barcode_length])

    if recipient_tag_grep is not None:
        if np.mean(fq[recipient_tag_grep.start() : recipient_tag_grep.end()]) >= min_qs:
            index_read = f_record[0][-17:].replace("+", "_")
            f_multitag = fr[f_multitag_pos : f_multitag_pos + 6]
            f_seqtag = fr[0:8]
            r_seqtag = rr[0:7]
            r_multitag = rr[r_multitag_pos : r_multitag_pos + 9]
            quality_bps_barcode_count += 1
            
            # Check whether in the reverse read, the four bases preceding oligo is CCGC
            oligo_r_prefix_grep = r_oligo_f_clipper.match(str(rr)[r_oligo_pos: r_oligo_pos + 4])
            
            if index_read in even_assembly_order:
                # Check whether in the forward read, the four bases preceding oligo is CGCC
                oligo_f_prefix_grep = f_oligo_f_clipper.match(str(fr)[f_oligo_pos_even: f_oligo_pos_even + 4])
                
                if oligo_f_prefix_grep is not None and oligo_r_prefix_grep is not None:
                    # check average quality score of both oligo regions 
                    if np.mean(fq[f_oligo_pos_even + 4 :]) >= min_qs and np.mean(rq[r_oligo_pos + 4 : r_oligo_pos + 4 + r_oligo_length]) >= min_qs:
                        vars()[index_read].write(recipient_tag_grep.group()[4:-4] + "," + str(fr)[f_oligo_pos_even + 4 :] + "," 
                                             + str(rr)[r_oligo_pos + 4 : r_oligo_pos + 4 + r_oligo_length] + ","
                                             + f_multitag + "," + r_multitag + "," + f_seqtag + r_seqtag + "," + "Even" + "\n")
            
            elif index_read in odd_assembly_order:            
                # First determine whether this is the 1st round or non-1st odd round
                # Check the expected position between 197 and 237 has the entry homology front and end sequences
                entry_homology_front_grep = entry_homology_front.match(str(fr)[entry_homology_front_pos : entry_homology_front_pos + entry_homology_length])
                entry_homology_end_grep = entry_homology_end.match(str(fr)[entry_homology_front_pos : entry_homology_front_pos + entry_homology_length][::-1])
                
                if entry_homology_front_grep is not None and entry_homology_end_grep is not None:
                    oligo_f_prefix_grep = f_oligo_f_clipper.match(str(fr)[f_oligo_pos_odd_1st: f_oligo_pos_odd_1st + 4])
                    if oligo_f_prefix_grep is not None and oligo_r_prefix_grep is not None:
                        if np.mean(fq[f_oligo_pos_odd_1st + 4 :]) >= min_qs and np.mean(rq[r_oligo_pos + 4 : r_oligo_pos + 4 + r_oligo_length]) >= min_qs:
                            vars()[index_read].write(recipient_tag_grep.group()[4:-4] + "," + str(fr)[f_oligo_pos_odd_1st + 4 :] + "," 
                                                     + str(rr)[r_oligo_pos + 4 : r_oligo_pos + 4 + r_oligo_length] + "," 
                                                     + f_multitag + "," + r_multitag + "," + f_seqtag + r_seqtag + "," + "Odd_1st" + "\n")
                
                elif entry_homology_front_grep is None and entry_homology_end_grep is None:
                    oligo_f_prefix_grep = f_oligo_f_clipper.match(str(fr)[f_oligo_pos_odd: f_oligo_pos_odd + 4])
                    if oligo_f_prefix_grep is not None and oligo_r_prefix_grep is not None:        
                        if np.mean(fq[f_oligo_pos_odd + 4 :]) >= min_qs and np.mean(rq[r_oligo_pos + 4 : r_oligo_pos + 4 + r_oligo_length]) >= min_qs:                                      
                            vars()[index_read].write(recipient_tag_grep.group()[4:-4] + "," + str(fr)[f_oligo_pos_odd + 4 :] + "," 
                                                     + str(rr)[r_oligo_pos + 4 : r_oligo_pos + 4 + r_oligo_length] + "," 
                                                     + f_multitag + "," + r_multitag + "," + f_seqtag + r_seqtag + "," + "Odd" + "\n")                                        
for i in oligo_indices:
    vars()[i].close()